# End-to-end AutoEmulate workflow 

## Overview

<!-- <b>In this workflow we demonstrate the integration of a Cardiovascular simulator, Naghavi Model from ModularCirc in an end-to-end AutoEmulate workflow.</b>  -->

Naghavi model is a 0D (zero-dimensional) computational model of the cardiovascular system, which is used to simulate blood flow and pressure dynamics in the heart and blood vessels.

This demo includes:
- Setting up parameter ranges 
- Creating samples
- Running the simulator to generate training data for the emulator 
- Using AutoEmulate to find the best pre-processing technique and model tailored to the simulation data 
- Applying history matching to refine the model and enhance parameter ranges 
- Sensitivity Analysis 
- Bayesian calibration

TODO: update figure?
<!-- <img src="https://raw.githubusercontent.com/alan-turing-institute/autoemulate/refs/heads/main/misc/workflow.png" alt="Work Flow" style="width:100%;"/> -->


In [1]:
import pandas as pd
import torch

## 1: Set up simulator and generate data

TODO: add link here to how subclass the `Simulator` class to integrate their own simulator into an `AutoEmulate` workflow (see `docs/experimental/tutorials/simulator/01_custom_simulations.ipynb`)

Below we import all that we need.

The `NaghaviSimulator` models a range of parameters. Below we choose to only track a subset of those. Note that the simulator is set up to output summary statistics for each of the tracked variables.

In [2]:
from cardiac_simulator import NaghaviSimulator

simulator = NaghaviSimulator(
    output_variables=['lv.P_i', 'lv.P_o'],  # Only the ones you're interested in
    n_cycles=300, 
    dt=0.001,
)

The simulator comes with predefined input parameters ranges. We can sample from those using Latin Hypercube Sampling to generate data to train the emulator with.

In [3]:
simulator.parameters_range

{'ao.r': (120.0, 360.0),
 'ao.c': (0.15, 0.44999999999999996),
 'art.r': (562.5, 1687.5),
 'art.c': (1.5, 4.5),
 'ven.r': (4.5, 13.5),
 'ven.c': (66.65, 199.95000000000002),
 'av.r': (3.0, 9.0),
 'mv.r': (2.05, 6.1499999999999995),
 'la.E_pas': (0.22, 0.66),
 'la.E_act': (0.225, 0.675),
 'la.v_ref': (5.0, 15.0),
 'la.k_pas': (0.01665, 0.07500000000000001),
 'lv.E_pas': (0.5, 1.5),
 'lv.E_act': (1.5, 4.5),
 'lv.v_ref': (5.0, 15.0),
 'lv.k_pas': (0.00999, 0.045)}

In [4]:
N_samples = 100
x = simulator.sample_inputs(N_samples)

We can now use the simulator to generate predictions for the sampled parameters. Alternatively, for convenience. we can load already simulated data.

In [5]:
run = False
save = True
read = True

if run:
    # Run batch simulations with the samples generated in Cell 1
    y = simulator.forward_batch(x)

    # Convert results to DataFrame for analysis
    results_df = pd.DataFrame(y)
    
    if save:
        # Save the results to a CSV file
        results_df.to_csv('simulator_results.csv', index=False)

if read:
    # Read the results from the CSV file
    results_df = pd.read_csv('simulator_results.csv')
    y = torch.tensor(results_df.to_numpy())

These are the output variables we've simulated.

In [6]:
simulator.output_names

['lv.P_i_min',
 'lv.P_i_max',
 'lv.P_i_mean',
 'lv.P_i_range',
 'lv.P_o_min',
 'lv.P_o_max',
 'lv.P_o_mean',
 'lv.P_o_range']

## 2: Train emulator with AutoEmulate
 
User should choose from the available `models` the `models` they would like to investigate. Here we restrict to just Gaussian Processes as we need an uncertainty aware emulator.

TODO: below we also use PCA to reduce dimensionality of the outputs but we should not do this if we're working with summary statistics.

In [18]:
from autoemulate.experimental.compare import AutoEmulate
from autoemulate.experimental.emulators.gaussian_process.exact import GaussianProcessExact
from autoemulate.experimental.transforms.pca import PCATransform

ae = AutoEmulate(
    x, 
    y, 
    models=[GaussianProcessExact], 
    y_transforms_list=[[PCATransform(n_components=2)]]
)

Comparing models: 100%|██████████| 1.00/1.00 [00:12<00:00, 12.9s/model]


TODO: the GPs are not doing great here

In [19]:
ae.summarise()

,model_name,x_transforms,y_transforms,config,rmse_test,r2_test,r2_test_std,r2_train,r2_train_std
0,GaussianProcessExact,[StandardizeTransform()],[PCATransform()],{'mean_module_fn': <function linear_mean at 0x...,3.798054,-0.246474,0.253701,0.517621,0.025239


Extract the best performing emulator.

In [20]:
model = ae.best_result().model

## 3: Sensitivity Analysis 
Use AutoEmulate to perform sensitivity analysis. This will help identify the parameters that have higher impact on the outputs to narrow down the search space for performing model calibration. 

TODO: could link to sensitivity analysis overview notebook.

In [21]:
from autoemulate.experimental.sensitivity_analysis import SensitivityAnalysis

# Define the problem dictionary for Sobol sensitivity analysis
problem = {
    'num_vars': simulator.in_dim,
    'names': simulator.param_names,
    'bounds': simulator.param_bounds
}

si = SensitivityAnalysis(model, problem=problem)

TODO: currently the below runs super slow, have no idea why. It crashed when I tried running it on 8 output variables.

In [15]:
# si_df = si.run(method='sobol')


DEBUG   2025-07-17 15:48:50,981 - autoemulate - Running sensitivity analysis with method=sobol, n_samples=1024, conf_level=0.95


In [ ]:
# si.plot_sobol(si_df)

## 4: Model calibration

To refine our emulator, we need real-world observations to compare against. These observations can come from experiments reported in the literature. 

In this example, we'll generate synthetic "observations" by running the simulator at the midpoint of each parameter range, treating these as our "ground truth" values for calibration. Note that in a real world example one can have multiple observations.

In [22]:
# An example of how to define observed data with means and variances from a hypothetical experiment
observations = {
    'lv.P_i_min': (5.0, 0.1),   # Minimum of minimum LV pressure
    'lv.P_i_max': (20.0, 0.1),   # Maximum of minimum LV pressure
    'lv.P_i_mean': (10.0, 0.1),  # Mean of minimum LV pressure
    'lv.P_i_range': (15.0, 0.5), # Range of minimum LV pressure
    'lv.P_o_min': (1.0, 0.1),  # Minimum of maximum LV pressure
    'lv.P_o_max': (13.0, 0.1),  # Maximum of maximum LV pressure
    'lv.P_o_mean': (12.0, 0.1), # Mean of maximum LV pressure
    'lv.P_o_range': (20.0, 0.5)  # Range of maximum LV pressure
}

In [12]:
# Otherwise, use one forward pass of your simualtion to get the observed data
# Calculate midpoint parameters
midpoint_params = []
for param_name, (min_val, max_val) in simulator.parameters_range.items():
    midpoint_params.append((min_val + max_val) / 2.0)
# Run the simulator with midpoint parameters
midpoint_results = simulator.forward(torch.tensor(midpoint_params).reshape(1, -1))

In [23]:
# Create observations dictionary
observations = {
    name: (val.item(), max(abs(val.item()) * 0.01, 0.01)) for
    name, val in 
    zip(simulator.output_names, midpoint_results[0])}
observations


{'lv.P_i_min': (12.257057189941406, 0.12257057189941406),
 'lv.P_i_max': (22.596208572387695, 0.22596208572387697),
 'lv.P_i_mean': (20.69025421142578, 0.20690254211425782),
 'lv.P_i_range': (10.339152336120605, 0.10339152336120605),
 'lv.P_o_min': (12.257057189941406, 0.12257057189941406),
 'lv.P_o_max': (22.596208572387695, 0.22596208572387697),
 'lv.P_o_mean': (20.69025421142578, 0.20690254211425782),
 'lv.P_o_range': (10.339152336120605, 0.10339152336120605)}

### History Matching
 
Once you have the final model, running history matching can improve your model. The Implausibility metric is calculated using the following relation for each set of parameter:

$I_i(\overline{x_0}) = \frac{|z_i - \mathbb{E}(f_i(\overline{x_0}))|}{\sqrt{\text{Var}[z_i - \mathbb{E}(f_i(\overline{x_0}))]}}$
Where if implosibility ($I_i$) exceeds a threshhold value, the points will be rulled out. 
The outcome of history matching are the NORY (Not Ruled Out Yet) and RO (Ruled Out) points.

- create a dictionary of your observations, this should match the output names of your simulator 
- create the history matching object 
- run history matching 


In [24]:
from autoemulate.experimental.calibration.history_matching import HistoryMatchingWorkflow

hmw = HistoryMatchingWorkflow(
    simulator=simulator,
    emulator=model,
    observations=observations,
    threshold=3.0,
    train_x=x.float(),
    train_y=y.float()
)

In [26]:
# Run history matching
test_parameters, impl_scores = hmw.run(n_simulations=20, n_test_samples=1000)

Running simulations: 100%|██████████| 20.0/20.0 [00:20<00:00, 1.01s/sample]


In [27]:
from autoemulate.history_matching_dashboard import HistoryMatchingDashboard
dashboard = HistoryMatchingDashboard(
    samples=test_parameters,
    impl_scores=impl_scores,
    param_names=simulator.param_names,  
    output_names=simulator.output_names, 
    )
dashboard.display()

HTML(value='<h2>History Matching Dashboard</h2>')

<img src="https://raw.githubusercontent.com/alan-turing-institute/autoemulate/refs/heads/main/misc/vis_dashboard_pic_sample.png" alt="Work Flow" style="width:20%;"/> """

#### 11 - MCMC
Once you have identified the important parameters through the Sensitivity analysis tool, the MCMC module can return the calibrated parameter values with uncertainty. 
The MCMC algorithm tries to find parameter values that match the predictions by the emulator to your `observations` whilst staying within the `parameters_range` (priors)
and accounting for uncertainty.





- Takes a pre-trained emulator (surrogate model)
- Uses sensitivity analysis results to identify the most important parameters
- Accepts observations (real data) to calibrate against
- Optionally incorporates NROY (Not Ruled Out Yet) samples from prior history matching
- Sets up parameter bounds for calibration

In [ ]:
from autoemulate.mcmc import MCMCCalibrator
# Define your observations (what you want to match)
# Define observed data with means and variances


# Run calibration
calibrator = MCMCCalibrator(
    emulator=gp_final,
    sensitivity_results=si,
    observations=observations,
    parameter_bounds=parameters_range,
    nroy_samples=nroy_samples,
    nroy_indices=nroy_indices,
    all_samples=all_samples,
    top_n_params=3  # Calibrate top 5 most sensitive parameters
)

results = calibrator.run_mcmc(num_samples=100, warmup_steps=10)
# Get calibrated parameter values
calibrated_params = calibrator.get_calibrated_parameters()
calibrated_params


In [ ]:
from autoemulate.mcmc_dashboard import MCMCVisualizationDashboard
dashboard = MCMCVisualizationDashboard(calibrator)
dashboard.display()

### Footnote: Testing the dashboard

Sometimes it is hard to know, if the results we are seeing is because the code is not working, or our simulation results are more interesting than we expected. Here is a little test dataset which tests the dashboard, so that you can see how the plots are supposed to look liek and what they shouldf show

In [ ]:
# Create a test sample with KNOWN NROY regions
test_samples = np.array([[x, y] for x in np.linspace(0,1,100) 
                               for y in np.linspace(0,1,100)])
test_scores = (abs(test_samples[:, 0]-0.5)+abs(test_samples[:, 1]-0.5)).reshape(-1, 1)

# Should show a clear diagonal pattern
test_dash = HistoryMatchingDashboard(
    samples=test_samples,
    impl_scores=test_scores,
    param_names=["p1", "p2"],
    output_names=["out1"],
    threshold=0.7  # ~50% of points should be NROY
)
#test_dash.display()